# Analiza zbioru danych 🎹 Spotify Tracks Dataset

W tym notatniku zaprezentowana zostanie analiza zbioru dannych [🎹 Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset). Najpierw przeprowadzona zostanie wstępna eksploracyjna analiza danych, a następnie wytrenowane zostaną modele, które przewidywać będą gatunek muzyki dla danej piosenki na podstawie atrybutów dostępnych w zbiorze danych.

### Import niezbędnych bibliotek oraz pakietów

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Opis zbioru danych

## Wczytanie zbioru danych oraz wstępne przetwarzanie

Z całego zbioru danych zostaną wyfiltrowane tylko niektóre gatunki, które nie zależą od pochodzenia/narodowości twórcy oraz które są popularne.

In [19]:
GENRES = ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'black-metal', 'blues', 'children', 'chill', 'classical', 'country', 'dance', 'death-metal', 'disco', 'edm', 'electro', 'funk', 'groove', 'hard-rock', 'hip-hop', 'jazz', 'opera', 'piano', 'pop', 'punk', 'reggae', 'rock-n-roll', 'rock', 'sad', 'sleep', 'soul', 'study']

Przykładowe 5 próbek ze zbioru danych

In [20]:
df = pd.read_csv("data/dataset.csv", index_col=0)
df = df[df['track_genre'].isin(GENRES)]
print(f"Liczba atrybutów: {len(df.columns)}")
df.sample(5)

Liczba atrybutów: 20


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
86485,5zTirec9tIsSVNgBSUmFxU,Charly García,Unplugged,Pasajera En Trance - Live,43,203973,False,0.5910,0.6140,0,-10.4920,1,0.0410,0.5270,0.0175,0.9790,0.6010,99.2800,4,punk
105918,33E1MSUblH1H3WdiaXEZL7,Justnormal,Growing Older,Early Mornings,16,123151,False,0.8160,0.4940,7,-10.2500,1,0.1010,0.8460,0.8800,0.0932,0.4030,93.7580,4,study
2900,507bYMYfbm6sUS9iEAaeSd,Seether;Amy Lee,Disclaimer II,Broken,70,258333,False,0.4480,0.6100,3,-2.6950,0,0.0342,0.0102,0.0005,0.1350,0.2140,123.7620,4,alt-rock
19318,6JZg74LR98YrRM0wlbCDLM,Rascal Flatts,Christmas Country Songs 2022,Joy To The World,0,218520,False,0.4170,0.8800,7,-4.0220,1,0.0521,0.1670,0.0000,0.0755,0.5790,149.9870,4,country
94711,1sMjYDNFpNoPjNH2ifBvuA,Beowülf;INTRN,Lose A Friend,Lose A Friend,48,202605,False,0.7560,0.3920,6,-10.1910,0,0.0446,0.2620,0.0000,0.1260,0.0658,125.0220,4,sad


## Opis zbioru danych

Zbiór danych posiada 20 atrybutów. Pogrubioną czcionką oznaczone zostały te, które będą używane w dalszej analizie danych oraz podczas uczenia modeli.
1. track_id - unikalny identyfikator utworu w ramach Spotify,
2. artists - zbiór artystów, którzy są autorami piosenki,
3. album_name - nazwa albumu, na którym pojawia się piosenka,
4. track_name - nazwa piosenki,
5. **popularity** - popularność piosenki wyrażona na skali od 0 do 100. Wyliczana jest ona na podstawie algorytmu przygotowanego przez Spotify, który bierze pod uwagę takie czynniki jak liczba odtworzeń utworu w serwisie oraz ich aktualności (czy utwór jest popularny teraz czy w przeszłości),
6. **duration_ms** - długość trwania pionseki w milisekundach,
7. **explicit** - wartość bool określająca czy piosenka posiada niecenzuralne słowa,
8. **danceability** - wartość od 0.0 do 1.0 określająca jak bardzo piosenka nadaje się do tańczenia. Wartość wyliczana jest między innymi na podstawie tempa czy też stabilności rytmu,
9. **energy** - wartość od 0.0 do 1.0 reprezentująca miarę intensywności utworu. Przykładowo piosenki, które są szybkie, głośne oraz hałaśliwe mają wysoką wartość energii,
10. **key** - tonacja, w której jest piosenka określona w notacji [Pitch class](https://en.wikipedia.org/wiki/Pitch_class),
11. **loudness** - średnia głośność piosenki wyrażona w decybelach,
12. **mode** - określenie czy piosenka jest w tonacji dur (1) czy moll (0),
13. **speechiness** - wartość od 0.0 do 1.0 odpowiadająca za obecność słów w piosence. Przykładowo poezja czytana albo podcast posiada wartość 1.0,
14. **acousticness** - wartość od 0.0 do 1.0 określająca jak bardzo akustyczny jest utwór,
15. **instrumentalness** - wartość od 0.0 do 1.0 określająca czy utwór zawiera słowa. Przykładowo piosenki, które są zaklasyfikowane jako Rap będą miały niską wartość instrumentalness. Im większa wartość tego atrybutu, tym większa pewność, że utwór nie posiada słów,
16. liveness - wartość od 0.0 do 1.0 określająca pewność z jaką utwór został wykonany na żywo,
17. **valence** - wartość od 0.0 do 1.0 określająca jak bardzo pozytywny jest utwór. Piosenki, które mają wartość 1.0 na tym atrybucie będą radosne, a z małymi wartościami (bliskimi 0.0) będą smutne,
18. **tempo** - średnia tempo utworu [BPM](https://en.wikipedia.org/wiki/Tempo),
19. time_signature - oszacowane taktowanie utworu,
20. **track_genre** - gatunek muzyki, do którego należy piosenka.

In [24]:
COLUMNS = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo', 'track_genre']
df = df[COLUMNS]

# Mapping key to object and mode to bool
df['key'] = df['key'].astype('category')
df['mode'] = df['mode'].astype('bool')

df.dtypes

popularity             int64
duration_ms            int64
explicit                bool
danceability         float64
energy               float64
key                 category
loudness             float64
mode                    bool
speechiness          float64
acousticness         float64
instrumentalness     float64
valence              float64
tempo                float64
track_genre           object
dtype: object

Większość z rozważanych atrybutów jest typu int lub float, czyli są to liczby natomiast są również atrybuty takie jak: *explicit* przyjmująca wartość boolowską - Prawda lub Fałsz, *key* która mimo tego, że określana jako integer ma charakter kategoryczny (nie można powiedzieć że próbka przyjmująca wartość 1 jest gorsza/lepsza od próbki przyjmującej wartość 2 na tym atrybucie), *mode* przyjmująca wartość boolowską - Prawda jeżeli tonacja dur, Fałsz jeżeli tonacja moll.

In [28]:
df.isnull().values.any()

False

W przygotowanym zbiorze danych nie ma wartości brakujących.

In [25]:
df.describe()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,tempo
count,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000,32000.0000
mean,32.1483,212219.9616,0.5554,0.5867,-9.2323,0.0756,0.3624,0.1729,0.4668,118.4999
std,26.4892,96688.2503,0.1934,0.2717,6.2607,0.0770,0.3538,0.3284,0.2655,31.2380
min,0.0000,17453.0000,0.0000,0.0000,-49.5310,0.0000,0.0000,0.0000,0.0000,0.0000
25%,1.0000,162897.0000,0.4280,0.3890,-10.8573,0.0356,0.0301,0.0000,0.2430,95.0040
50%,33.0000,200299.0000,0.5790,0.6220,-7.3095,0.0477,0.2230,0.0000,0.4600,118.2930
75%,55.0000,244053.0000,0.7020,0.8160,-5.2530,0.0791,0.7030,0.0826,0.6810,137.6435
max,100.0000,4120258.0000,0.9800,1.0000,1.8210,0.9430,0.9960,1.0000,0.9950,243.3720


W powyższej tabeli przedstawione zostały statystyki dotyczące każdego z atrybutów liczbowych. 

# Interpretacja przypadków ze zbioru danych

# Eksploracyjna analiza danych

# Modele uczenia maszynowego

# Interpretacja modeli

# Źródła

# Wykorzystane biblioteki oraz pakiety